In [1]:
import sys
import torch
sys.path.insert(0, '../../Models')
sys.path.insert(0, '../../Utils')
from _utils import  sample_random_glue_stsb, collect_info_for_metric, save_info, \
                    get_continuation_mapping, get_continuous_attributions, get_continuous_raw_inputs,\
                    attr_normalizing_func
from preload_models import get_stsb_tokenizer_n_model

In [2]:
stsb_data_raw, targets, idxs = sample_random_glue_stsb()

Reusing dataset glue (/home/user/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

[ 436  138  931 1462 1491  577  873  899  268  603 1341  648  941  901
  335 1306  234  132 1257 1433 1351 1484  977  146 1275  225  396  757
  949 1151  605 1041  350  788  626   88  599 1080  882  927 1209  647
  486  214  317  895  897   50 1054 1454]


In [3]:
tokenizer, model = get_stsb_tokenizer_n_model()

In [4]:
#define some containers to save some info
model_out_list, raw_attr_list, conti_attr_list, raw_input_list = [], [], [], []

In [5]:
from captum.attr import IntegratedGradients
from captum.attr import visualization 

In [6]:
IG = IntegratedGradients(model)

In [7]:
def generate_record(raw_datum, target): #raw_datum expected to be a tuple/list of 2 sentences
    #tokenizer operations
    tokenized = tokenizer(raw_datum, truncation=True, return_offsets_mapping=True)
    offset_mappings = tokenized['offset_mapping']
    #concatenate the two offset_mappings together because they are fed in together
    conti_map = get_continuation_mapping(offset_mappings[0]) + get_continuation_mapping(offset_mappings[1])
    #change the first input_id of the second sentence to be the last input_id of the 1st sentence (i.e. an [END] token))
    tokenized_input_ids = tokenized['input_ids'][0] + \
                        [tokenized['input_ids'][1][i] if i != 0 else tokenized['input_ids'][0][-1] \
                         for i in range(len(tokenized['input_ids'][1]))]
    input_ids = torch.tensor(tokenized_input_ids).unsqueeze(0)
    detokenized = [t.replace('Ġ', '') for t in tokenizer.convert_ids_to_tokens(input_ids[0])]
    
    #feeding input forward 
    input_emb = model.get_embeddings(input_ids)
    pred_prob = model(input_emb).item()
    print(f'pred_prob {pred_prob*5}')

     #categorizing results
    pred_class = 'Similar' if pred_prob > 0.5 else 'Not Similar' 
    true_class = 'Similar' if target > 2.5 else 'Not Similar' 
    
    #attribution algorithm working
    attribution, delta = IG.attribute(input_emb, return_convergence_delta=True)
    word_attributions = attribution.squeeze(0).sum(dim=1)
    word_attributions = attr_normalizing_func(word_attributions.to(torch.float32))
    word_attributions /= torch.norm(word_attributions)
    attr_score = torch.sum(word_attributions)
    attr_class = 'Similar' if attr_score > 0.5 else 'Not Similar'
    convergence_score = delta
    
    
#     #re-organizing tensors and arrays because words get split down
    conti_attr = get_continuous_attributions(conti_map, word_attributions)
    raw_input = get_continuous_raw_inputs(conti_map, detokenized)

#     print(f'word attributions {word_attributions}')
#     print(f'pred_prob {pred_prob}')
#     print(f'pred_class {pred_class}')
#     print(f'true_class {true_class}')
#     print(f'attribution {attribution}')
#     print(f'attr_class {attr_class}')
#     print(f'attr_score {attr_score}')
#     print(f'raw_input {raw_input}')

        
# #     collect info for metrics later
    collect_info_for_metric(model_out_list, pred_prob, raw_attr_list, attribution, conti_attr_list, conti_attr, raw_input_list, raw_input)
        
    
    visual_record = visualization.VisualizationDataRecord(word_attributions=conti_attr,
                                                         pred_prob=pred_prob,
                                                         pred_class=pred_class,
                                                         true_class=true_class,
                                                         attr_class=attr_class,
                                                         attr_score=attr_score,
                                                         raw_input=raw_input,
                                                         convergence_score=convergence_score)
        
        
    return visual_record
      
    

In [8]:
for i, (datum_raw, target) in enumerate(zip(stsb_data_raw, targets), start=1):
#     example_1 = 'The man cut down a tree with an axe.'
#     example_2 = 'A man chops down a tree with an axe.'
#     datum_raw, target = [example_1, example_1], 1
    print(f'Raw review: {datum_raw}') #datum expected to be a list of 2 sentences
    print(f'GT target: {target}')
    visual_record=generate_record(datum_raw, target)
    print(visualization.visualize_text([visual_record]))   

Raw review: ['Two dogs are sniffing something in the rocks.', 'Two dogs running down a path in the woods.']
GT target: 1.600000023841858
pred_prob 2.0290477573871613
attr dtype torch.float32
word attr tensor([ 5.6297e-01,  3.4591e-02,  4.0276e-02,  4.1100e-02,  2.6408e-02,
         4.3208e-04, -1.2707e-01, -6.0463e-02, -2.5766e-01, -1.1550e-01,
         2.3158e-02,  1.2136e-02, -2.1563e-01,  3.2570e-02,  3.7114e-02,
         7.9196e-03,  5.1936e-03, -2.8277e-01, -2.6309e-01, -2.3793e-01,
        -5.6297e-01,  8.5040e-03,  3.3206e-03, -7.6265e-02],
       grad_fn=<DivBackward0>)
conti attr [tensor(0.5630, grad_fn=<UnbindBackward0>), tensor(0.0346, grad_fn=<UnbindBackward0>), tensor(0.0403, grad_fn=<UnbindBackward0>), tensor(0.0411, grad_fn=<UnbindBackward0>), tensor(0.0268, grad_fn=<AddBackward0>), tensor(-0.1271, grad_fn=<UnbindBackward0>), tensor(-0.0605, grad_fn=<UnbindBackward0>), tensor(-0.2577, grad_fn=<UnbindBackward0>), tensor(-0.0923, grad_fn=<AddBackward0>), tensor(0.0121, gra

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Not Similar,Not Similar (0.41),Not Similar,-1.36,#s Two dogs are sniffing something in the rocks. #/s #/s Two dogs running down a path in the woods. #/s


<IPython.core.display.HTML object>
Raw review: ['A woman is slicing a potato.', 'A woman is slicing carrot.']
GT target: 2.5
pred_prob 2.1180687844753265
attr dtype torch.float32
word attr tensor([ 0.6317,  0.0098,  0.1079,  0.0030,  0.0264, -0.6317, -0.0519, -0.0618,
        -0.2664, -0.2450,  0.0752,  0.0863, -0.0216, -0.0093, -0.1476, -0.1255,
        -0.0272], grad_fn=<DivBackward0>)
conti attr [tensor(0.6317, grad_fn=<UnbindBackward0>), tensor(0.0098, grad_fn=<UnbindBackward0>), tensor(0.1079, grad_fn=<UnbindBackward0>), tensor(0.0030, grad_fn=<UnbindBackward0>), tensor(0.0264, grad_fn=<UnbindBackward0>), tensor(-0.6317, grad_fn=<UnbindBackward0>), tensor(-0.1137, grad_fn=<AddBackward0>), tensor(-0.2664, grad_fn=<UnbindBackward0>), tensor(-0.2450, grad_fn=<UnbindBackward0>), tensor(0.0752, grad_fn=<UnbindBackward0>), tensor(0.0863, grad_fn=<UnbindBackward0>), tensor(-0.0216, grad_fn=<UnbindBackward0>), tensor(-0.0093, grad_fn=<UnbindBackward0>), tensor(-0.2732, grad_fn=<AddBackwar

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Not Similar,Not Similar (0.42),Not Similar,-0.65,#s A woman is slicing a potato. #/s #/s A woman is slicing carrot. #/s


<IPython.core.display.HTML object>
Raw review: ['It was before the season began because Amy saw the woman before the Silence took her.', "The exact timeline hasn't been listed yet, but there are a few assumptions we can make."]
GT target: 0.20000000298023224
pred_prob 0.16538595780730247
attr dtype torch.float32
word attr tensor([ 0.3198, -0.2166,  0.0360, -0.3198, -0.1204, -0.2808, -0.1053,  0.0100,
        -0.2224, -0.2508, -0.1826, -0.0943, -0.2396, -0.0842,  0.0019, -0.1446,
        -0.0145, -0.0266, -0.0335, -0.0177,  0.0010,  0.0042, -0.0235, -0.1060,
        -0.3034, -0.2018, -0.1267, -0.2928, -0.1772, -0.1726, -0.1089, -0.1968,
        -0.0932, -0.0168, -0.0331, -0.1024, -0.0433, -0.0846, -0.1118,  0.0130],
       grad_fn=<DivBackward0>)
conti attr [tensor(0.3198, grad_fn=<UnbindBackward0>), tensor(-0.2166, grad_fn=<UnbindBackward0>), tensor(0.0360, grad_fn=<UnbindBackward0>), tensor(-0.3198, grad_fn=<UnbindBackward0>), tensor(-0.1204, grad_fn=<UnbindBackward0>), tensor(-0.2808

...

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Not Similar,Not Similar (0.14),Not Similar,-1.16,#s Two little girls are talking on the phone. #/s #/s A little girl is walking down the street. #/s


<IPython.core.display.HTML object>
Raw review: ["The report shows that drugs sold in Canadian pharmacies are manufactured in facilities approved by Health Canada - the FDA's counterpart in Canada.", 'The report shows that drugs sold in Canadian pharmacies are manufactured in facilities approved by Health Canada, which serves a similar role as the FDA for the Canadian government.']
GT target: 5.0
pred_prob 4.7438859939575195
attr dtype torch.float32
word attr tensor([ 4.4388e-01, -3.4245e-02, -3.5280e-03, -7.6760e-02, -9.5338e-02,
        -1.6372e-02, -7.0271e-03, -1.4721e-01,  1.2039e-02, -1.4658e-02,
        -7.5777e-02,  3.9557e-03, -3.4466e-01, -5.7639e-02, -3.5311e-02,
        -8.4222e-02,  2.0441e-03,  1.2047e-03, -4.7609e-03, -8.6929e-02,
         8.7073e-03, -9.9811e-02, -3.2685e-02, -7.4735e-02,  3.1871e-04,
        -2.8064e-01, -6.1292e-02, -4.4192e-02,  7.4405e-03,  7.5860e-04,
         2.4090e-03, -2.3720e-01,  3.1004e-04,  2.2301e-03, -2.9639e-01,
        -1.5585e-01,  4.03

<IPython.core.display.HTML object>
Raw review: ["Foreign companies eye new 'opening-up'", 'More carriers likely in China']
GT target: 0.6000000238418579
pred_prob 0.07349737454205751
attr dtype torch.float32
word attr tensor([ 0.4392,  0.0923,  0.0580,  0.0716, -0.1052, -0.1170,  0.0548,  0.0289,
         0.0016, -0.3002, -0.4136, -0.4041, -0.0319,  0.0103,  0.0234, -0.3679,
         0.0837, -0.4392], grad_fn=<DivBackward0>)
conti attr [tensor(0.4392, grad_fn=<UnbindBackward0>), tensor(0.0923, grad_fn=<UnbindBackward0>), tensor(0.0580, grad_fn=<UnbindBackward0>), tensor(0.0716, grad_fn=<UnbindBackward0>), tensor(-0.1052, grad_fn=<UnbindBackward0>), tensor(-0.3320, grad_fn=<AddBackward0>), tensor(-0.4136, grad_fn=<UnbindBackward0>), tensor(-0.4041, grad_fn=<UnbindBackward0>), tensor(-0.0319, grad_fn=<UnbindBackward0>), tensor(0.0103, grad_fn=<UnbindBackward0>), tensor(0.0234, grad_fn=<UnbindBackward0>), tensor(-0.3679, grad_fn=<UnbindBackward0>), tensor(0.0837, grad_fn=<UnbindBackward0>

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Not Similar,Not Similar (0.01),Not Similar,-1.32,#s Foreign companies eye new 'opening-up' #/s #/s More carriers likely in China #/s


<IPython.core.display.HTML object>


In [9]:
save_info(idxs, stsb_data_raw, targets, model_out_list, raw_attr_list, conti_attr_list, raw_input_list, fname='integrated_gradients_out.pkl')

{'indices': [436,
  138,
  931,
  1462,
  1491,
  577,
  873,
  899,
  268,
  603,
  1341,
  648,
  941,
  901,
  335,
  1306,
  234,
  132,
  1257,
  1433,
  1351,
  1484,
  977,
  146,
  1275,
  225,
  396,
  757,
  949,
  1151,
  605,
  1041,
  350,
  788,
  626,
  88,
  599,
  1080,
  882,
  927,
  1209,
  647,
  486,
  214,
  317,
  895,
  897,
  50,
  1054,
  1454],
 'raw_data': [['Two dogs are sniffing something in the rocks.',
   'Two dogs running down a path in the woods.'],
  ['A woman is slicing a potato.', 'A woman is slicing carrot.'],
  ['It was before the season began because Amy saw the woman before the Silence took her.',
   "The exact timeline hasn't been listed yet, but there are a few assumptions we can make."],
  ['Protests for kidnapped girls banned in Nigerian capital',
   "Scores 'killed' in Boko Haram raid in Nigeria"],
  ['Oil falls in Asian trade', 'Oil prices down in Asian trade'],
  ['There are people out on the street.', 'People are out on the street.'],
 